<a href="https://colab.research.google.com/github/piusAI/YOLO/blob/main/YOLOTrain_STUD01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# Before

In [ ]:
from copy import copy
from pathlib import Path
from typing import Any, Dict, Optional, Union

from ultralytics.models import yolo
from ultralytics.nn.tasks import PoseModel
import ultralytics
from ultralytics.utils import DEFAULT_CFG, LOGGER
from ultralytics.utils.plotting import plot_results

# yolo = ultralytics.models.yolo
# PoseModel = ultralytics.nn.task.PoseModel

In [ ]:
class PoseTrainer(yolo.detect.DetectionTrainer):
  def __init__(self, cfg=DEFAULT_CFG, overrides : Optional[Dict[str,Any]] = None, _callbakcs=None):
    if overrides is None:
      overrides = {}
      overrides["task"] = 'pose'
      super().__init__(cfg, overrides, _callbacks)

    if isinstance(self.args.device, str) and self.args.device.lower() =='mps':
      LOGGER.warning(
          "Apple MPS Known Pose bug. Recommend 'device=cpu', for Pose models."
          "See YOLO Issue-4031"
      )
  def get_model(
      self,
      cfg : Optional[Union[str,Path, Dict[str, Any]]] = None,
      weights : Optional[Union[str, Path]] = None,
      verbose : bool = True,

  ) -> PoseModel:
    model = PoseModel(
        cfg, nc=self.data['nc'], ch = self.data['channels'], data_kpt_shape=self.data['kpt_shape'], verbose = verbose
    )
    if weights:
      model.load(weights)

      return model

BaseTrainer !

In [ ]:
"""
Train a model on a dataset.

Usage:
    $ yolo mode=train model=yolo11n.pt data=coco8.yaml imgsz=640 epochs=100 batch=16
"""

import gc
import math
import os
import subprocess
import time
import warnings
from copy import copy, deepcopy
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import torch
from torch import distributed as dist
from torch import nn, optim

from ultralytics import __version__
from ultralytics.cfg import get_cfg, get_save_dir
from ultralytics.data.utils import check_cls_dataset, check_det_dataset
from ultralytics.nn.tasks import attempt_load_one_weight, attempt_load_weights
from ultralytics.utils import (
    DEFAULT_CFG,
    LOCAL_RANK,
    LOGGER,
    RANK,
    TQDM,
    YAML,
    callbacks,
    clean_url,
    colorstr,
    emojis,
)
from ultralytics.utils.autobatch import check_train_batch_size
from ultralytics.utils.checks import check_amp, check_file, check_imgsz, check_model_file_from_stem, print_args
from ultralytics.utils.dist import ddp_cleanup, generate_ddp_command
from ultralytics.utils.files import get_latest_run
from ultralytics.utils.torch_utils import (
    TORCH_2_4,
    EarlyStopping,
    ModelEMA,
    autocast,
    convert_optimizer_state_dict_to_fp16,
    init_seeds,
    one_cycle,
    select_device,
    strip_optimizer,
    torch_distributed_zero_first,
    unset_deterministic,
)

In [ ]:
class BaseTrainer:
  def __init__(self, cfg = DEFAULT_CFG, overrides=None, _callbacks=None ):
    self.args = get_cfg()

# BasePredictor


In [3]:
import platform
import re, threading
from pathlib import Path
from typing import Any, Dict, List, Optional

import cv2
import numpy as np
import torch

from ultralytics.cfg import get_cfg, get_save_dir
from ultralytics.data import load_inference_source
from ultralytics.data.augment import LetterBox
from ultralytics.nn.autobackend import AutoBackend
from ultralytics.utils import DEFAULT_CFG, LOGGER, MACOS, WINDOWS, callbacks, colorstr, ops
from ultralytics.utils.checks import check_imgsz, check_imshow
from ultralytics.utils.files import increment_path
from ultralytics.utils.torch_utils import select_device, smart_inference_mode

In [ ]:
class BasePredictor_Stud:
  def __init__(
      self,
      cfg = DEFAULT_CFG,
      overrides : Optional[Dict[str, Any]] = None, #override
      # Ex : {'conf' : 0, 'iou' : 0.6, 'cee':0.2} || None
      _callbacks : Optional[Dict[str, List[callable]]]= None,
  ) :


## Callback?

In [3]:
!pip install typing

In [14]:
from typing import Optional, Dict, List, Any, Callable


# Declare Callback function
# predictor instance 매개변수 받겠다 선언 - Callback은 서로를 REF
def my_on_predict_start(Predictor_instance):
  print("-- CUSTOM CALL BACK : Predicting Started! ---")
  # Predictor_instance.args.verbose =True =>필요시 predictor의 속성 변경 가능

def my_on_predict_batch_end(Predictor_instance):
  if hasattr(Predictor_instance, 'results') and Predictor_instance.results:
    speed_info = Predictor_instance.results[0]['speed'] if Predictor_instance.results else{}
    print(f'-- CUSTOM CALL BACK : Batch processed. Speed: {speed_info} --')

def my_on_predict_end(Predictor_instance):
  print('\n--- CUSTOM CALL BACK : Predicting finished! ---')


''' --------------------------------------------- '''

# _callbacks dictionary
custom_callbacks_dict : Dict[str, List[Callable[[Any],None]]] = {
    "on_predict_start": [my_on_predict_start],
    "on_predict_batch_end" : [my_on_predict_batch_end],
    "on_predict_end" : [my_on_predict_end],
}

#Base_studd사용, 초기화
class BasePredictor_Studd:
  def __init__(
      self,
      cfg = "DEFAULT_CFG_VALUE",
      overrides : Optional[Dict[str, Any]] = None,
      _callbacks : Optional[Dict[str, List[Callable]]] = None,
  ):
      self.cfg = cfg
      self.overrides = overrides
      self.callbacks = _callbacks if _callbacks is not None else{} #None일때, 빈 딕셔너리
      ## 실제 YOLO는 Default_Callback을 불러옴

  def run_callbacks(self, event : str):
    '''
    특정 event에 등록된 callback 실행!
    따로 지정해놓은 Callback dictionary를 가져옴
    '''
    if event in self.callbacks:
      for callback_func in self.callbacks[event]:
        callback_func(self) #predictor instance 자신을 callback으로 다시 받음

  def simulate_predict(self):
    self.run_callbacks('on_predict_start')
    print('\n(Simulating batch 1 processing..)')
    # 실제는 results[0], speed가속성이 update 될것임.
    self.results = [{'speed' : {'preprocess' : 10, 'inference' : 50, 'postprocess': 5}}]

    self.run_callbacks('on_predict_batch_end')

    print('\n(Simulating batch 2 processing...)')
    self.results = [{'speed' : {'preprocess' : 12, 'inference' : 55, 'postprocess': 6}}]
    self.run_callbacks('on_predict_batch_end')

    self.run_callbacks('on_predict_end')


my_predictor = BasePredictor_Studd(_callbacks=custom_callbacks_dict)
#Base Predictor_stud instance생성이후, custom callback dic전달.

my_predictor.simulate_predict()
#가상 loop 실행

-- CUSTOM CALL BACK : Predicting Started! ---

(Simulating batch 1 processing..)
-- CUSTOM CALL BACK : Batch processed. Speed: {'preprocess': 10, 'inference': 50, 'postprocess': 5} --

(Simulating batch 2 processing...)
-- CUSTOM CALL BACK : Batch processed. Speed: {'preprocess': 12, 'inference': 55, 'postprocess': 6} --

--- CUSTOM CALL BACK : Predicting finished! ---
